![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForTokenClassification.ipynb)


## Import ONNX BertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `BertForTokenClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for token classification via `BertForTokenClassification` or `TFBertForTokenClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFBertForTokenClassification](https://huggingface.co/transformers/model_doc/bert.html#tfbertfortokenclassification)
- Some [example models](https://huggingface.co/models?filter=bert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [dslim/bert-base-NER](https://huggingface.co/dslim/bert-base-NER) model from HuggingFace as an example
- In addition to the BERT model, we also need to save the `BertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import BertTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification

MODEL_NAME = "dslim/bert-base-NER"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The model dslim/bert-base-NER was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

('onnx_models/dslim/bert-base-NER/tokenizer_config.json',
 'onnx_models/dslim/bert-base-NER/special_tokens_map.json',
 'onnx_models/dslim/bert-base-NER/vocab.txt',
 'onnx_models/dslim/bert-base-NER/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {EXPORT_PATH}

total 421332
-rw-r--r-- 1 root root       999 Jun  9 21:32 config.json
-rw-r--r-- 1 root root 431208242 Jun  9 21:32 model.onnx
-rw-r--r-- 1 root root       125 Jun  9 21:32 special_tokens_map.json
-rw-r--r-- 1 root root      1291 Jun  9 21:32 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun  9 21:32 vocab.txt


- We need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
!mkdir {EXPORT_PATH}/assets

labels = ort_model.config.label2id
labels = sorted(labels, key=labels.get)

with open(EXPORT_PATH +'/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [ ]:
!ls -lR {EXPORT_PATH}

onnx_models/dslim/bert-base-NER:
total 421124
drwxr-xr-x 2 root root      4096 Jun  9 21:32 assets
-rw-r--r-- 1 root root       999 Jun  9 21:32 config.json
-rw-r--r-- 1 root root 431208242 Jun  9 21:32 model.onnx
-rw-r--r-- 1 root root       125 Jun  9 21:32 special_tokens_map.json
-rw-r--r-- 1 root root      1291 Jun  9 21:32 tokenizer_config.json

onnx_models/dslim/bert-base-NER/assets:
total 216
-rw-r--r-- 1 root root     51 Jun  9 21:32 labels.txt
-rw-r--r-- 1 root root 213450 Jun  9 21:32 vocab.txt


In [ ]:
!cat {EXPORT_PATH}/assets/labels.txt

O
B-MISC
I-MISC
B-PER
I-PER
B-ORG
I-ORG
B-LOC
I-LOC

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save BertForTokenClassification in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [ ]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 28.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.




In [ ]:
from sparknlp.annotator import BertForTokenClassification

tokenClassifier = BertForTokenClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your BertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 421180
-rw-r--r-- 1 root root 431274176 Jun  9 21:35 bert_classification_onnx
drwxr-xr-x 4 root root      4096 Jun  9 21:35 fields
drwxr-xr-x 2 root root      4096 Jun  9 21:35 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForTokenClassification model 😊

In [ ]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'B-MISC', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example = spark.createDataFrame([
    ["Barack Obama was born in Hawaii and served as the 44th President of the United States."],
    ["Google was founded in September 1998 by Larry Page and Sergey Brin in California."],
    ["The Eiffel Tower is located in Paris and was completed in 1889."],
]).toDF("text")

result = pipeline.fit(example).transform(example)
result.select("text", "ner.result").show(truncate=False)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------+
|text                                                                                  |result                                                                 |
+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------+
|Barack Obama was born in Hawaii and served as the 44th President of the United States.|[B-PER, I-PER, O, O, O, B-LOC, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O]|
|Google was founded in September 1998 by Larry Page and Sergey Brin in California.     |[B-ORG, O, O, O, O, O, O, B-PER, I-PER, O, B-PER, I-PER, O, B-LOC, O]  |
|The Eiffel Tower is located in Paris and was completed in 1889.                       |[O, B-LOC, I-LOC, O, O, O, B-LOC, O, O, O, O, O, O]                    |
+---------------------------------

That's it! You can now go wild and use hundreds of `BertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
